In [6]:
import ee
import geemap
import geopandas as gpd
from ipyleaflet import GeoJSON
from zipfile import ZipFile
import numpy as np
import os
import pandas as pd

ee.Initialize()

i_date = '2022-12-30'
f_date = '2022-12-31'
s5p_layer = 'CH4_column_volume_mixing_ratio_dry_air_bias_corrected'
s5p_methane_dataset = ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_CH4")
s5p_metane = s5p_methane_dataset.select(s5p_layer).filterDate(i_date, f_date)

countries = ee.FeatureCollection('users/hmojica/data/countries')

colombia_shp = countries.filterMetadata('COUNTRY', 'equals', 'Colombia')
#s5p_metane_img = s5p_metane.map(lambda image: image.clip(colombia_shp))
#s5p_metane_dpto = s5p_metane_img.filterBounds(colombia_shp)
aa = colombia_shp.geometry()
s5p_metane_dpto = s5p_metane.filterBounds(aa)
s5p_metane_img = s5p_metane_dpto.map(lambda image: image.clip(aa))


#s5p_metane_dpto.getInfo()
#s5p_metane_img.toDictionary(['date_range', 'period', 'product_tags', 'sample'])
#np.array(s5p_metane_img.getInfo())




In [ ]:
def ee_array_to_df(arr, list_of_bands):
    """Transforms client-side ee.Image.getRegion array to pandas.DataFrame.
    Modification of 
    https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api-guiattard
    """
    df = pd.DataFrame(arr)

    # Rearrange the header.
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Remove rows without data inside.
    df = df[['longitude', 'latitude', 'time', *list_of_bands]].dropna()

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors='coerce')

    # Convert the time field into a datetime.
    df['datetime'] = pd.to_datetime(df['time'], unit='ms')
    # Keep the columns of interest.
    df = df[['datetime', "longitude", "latitude", *list_of_bands]]

    return df


df_area_of_interest = ee_array_to_df(s5p_metane_img.getInfo(), [s5p_layer])

# to give you a preview
# df_area_of_interest.head()

# Plot the spatial map using the latitude and longitude (in practice, you need to pick specific date by doing df_area_of_interest[df_area_of_interest["datetime"]=="2003-06-10"]
plt.scatter(df_area_of_interest["longitude"], df_area_of_interest["latitude"], c=df_area_of_interest["NDVI"])
plt.colorbar()
#out_dir = os.path.join('/home/bojack/Downloads/colombia/data')
#s5p_metane_dpto.aggregate_array('system:index').getInfo()
#geemap.ee_export_image_collection(s5p_metane_dpto, out_dir=out_dir)


#html_file = os.path.join(out_dir, 'my_map.html')
#Map.to_html(filename=html_file, title='My Map', width='100%', height='880px')

{'type': 'ImageCollection', 'bands': [], 'id': 'COPERNICUS/S5P/OFFL/L3_CH4', 'version': 1676492845711580.0, 'properties': {'system:visualization_0_min': '1750.0', 'type_name': 'ImageCollection', 'keywords': ['ch4', 'climate', 'copernicus', 'esa', 'eu', 'knmi', 'methane', 's5p', 'sentinel', 'sron', 'tropomi'], 'thumb': 'https://mw1.google.com/ges/dd/images/COPERNICUS_S5P_OFFL_CH4_thumb.png', 'description': '<h3 id="offll3_ch4">OFFL/L3_CH4</h3><p>This dataset provides offline high-resolution imagery of methane\nconcentrations.</p><p>Methane (CH<sub>4</sub>) is, after carbon dioxide (CO<sub>2</sub>), the most important contributor\nto the anthropogenically enhanced greenhouse effect. Roughly three-quarters\nof methane emissions are anthropogenic and as such it is important to continue\nthe record of satellite based measurements. TROPOMI aims at providing CH<sub>4</sub>\ncolumn concentrations with high sensitivity to the Earth’s surface, good\nspatiotemporal coverage, and sufficient accura

In [ ]:
s5p_methane_viz = {
    'min': 1600,
    'max': 2000,
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red'],
    'opacity': 1.0
}
Map = geemap.Map()

Map.addLayer(s5p_metane_img, s5p_methane_viz, 'methane in Colombia ')



Map